In [ ]:
PCHOME_PROD_LINK_URL = 'https://24h.pchome.com.tw/prod/'
PCHOME_PIC_URL = 'https://d.ecimg.tw'
header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'Connection': 'close'
}

PROD_NAME = 'ssd'
page_num = 2
PCHOME_URL = 'https://ecshweb.pchome.com.tw/search/v3.3/all/results?q='+ PROD_NAME +'&page='+ str(page_num) +'&sort=sale/dc'
resp = requests.get(PCHOME_URL, headers=header)

In [ ]:
data = json.loads(resp.text)
prods = data['prods']
print(prods)

In [ ]:
# pchome crawler func
# SEARCH_NUM: 表20個

import requests
import json
import time

def pchome_crawler(PROD_NAME, SEARCH_NUM):
    # base
    PCHOME_PROD_LINK_URL = 'https://24h.pchome.com.tw/prod/'
    PCHOME_PIC_URL = 'https://d.ecimg.tw'
    header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
        'Connection': 'close'
    }
    
    print(PROD_NAME)
    print(SEARCH_NUM)
    
    # 存放最後clean出來的商品資料
    prod_list = [] 
    
    
    
    for page_num in range(1, int(SEARCH_NUM/20)+1):
        PCHOME_URL = 'https://ecshweb.pchome.com.tw/search/v3.3/all/results?q='+ PROD_NAME +'&page='+ str(page_num) +'&sort=sale/dc'
        resp = requests.get(PCHOME_URL, headers=header)
        print(resp.status_code)
        print('zz')
        if resp.status_code == 200: # 如果請求成功，則去抓產品列表
            data = json.loads(resp.text)
            prods = data['prods']
            print(prods)

            for item in prods: # 抓細項
                prod_id = item['Id']
                prod_name = item['name']
                prod_price = str(item['price'])
                prod_url = PCHOME_PROD_LINK_URL + prod_id
                prod_pic = PCHOME_PIC_URL + item['picS']

                obj = {
                    'from': 'pchome',
                    'id': prod_id,
                    'name': prod_name,
                    'prod_price': prod_price,
                    'url': prod_url,
                    'prod_pic': prod_pic
                }
                # q.put(obj)
                print(obj)
                prod_list.append(obj)

            time.sleep(0.5) 
        else:
            print('error')
            break
    
    return prod_list

In [ ]:
# main
for i in range(1, int(40/20)+1):
    print(i)

In [ ]:
pchome_crawler('ssd', 10)

In [ ]:
# u-mall crawler func

import requests
import json
import time

def umall_crawler(q, PROD_NAME, SEARCH_NUM):
    # base
    UMALL_URL = 'https://www.u-mall.com.tw/Search/Get'
    UMALL_PROD_LINK_URL = 'https://www.u-mall.com.tw/'
    header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
        'Connection': 'close'
    }
    
    # 存放最後clean出來的商品資料
    # prod_list = [] 
    
    # main
    POST_HEADER = {
        'keyword': PROD_NAME,
        'model[cateName]': '全站',
        'model[pageSize]': int(SEARCH_NUM)
    }
    resp = requests.post(UMALL_URL, headers=header, data=POST_HEADER)
    
    if resp.status_code == 200:
        ori_data = json.loads(resp.text)
        data = ori_data['searchResult']['products']

        for item in data:
            prod_id = str(item['id'])
            prod_name = item['title']
            prod_price = item['finalPrice']
            prod_url = UMALL_PROD_LINK_URL + item['pageLink']
            prod_pic = 'https:' + item['imageUrl']

            obj = {
                'from': 'umall',
                'id': prod_id,
                'name': prod_name,
                'prod_price': prod_price,
                'url': prod_url,
                'prod_pic': prod_pic
            }
            q.put(obj)
            # prod_list.append(obj)
            
        # return prod_list
    else:
        print('error')

In [ ]:
umall_crawler('ssd', 100)

## 平行處理 (Multi Thread)

In [ ]:
import threading

def get_pchome_data():
    data = pchome_crawler('ssd', 100)
#     for item in data:
#         data_pool = data_pool.append(item)
    print(data)
    
def get_umall_data():
    data = umall_crawler('ssd', 100)
#     for item in data:
#         data_pool = data_pool.append(item)
    print(data)
    

t_pchome_crawler = threading.Thread(target = get_pchome_data)
t_umall_crawler = threading.Thread(target = get_umall_data)

t_pchome_crawler.start()
t_umall_crawler.start()

t_pchome_crawler.join()
t_umall_crawler.join()

# 包成class

In [ ]:
import queue
import threading

In [ ]:
class crawler:
    def __init__(self, *crawler_algo):
        self.crawler_algo = crawler_algo # 存各ec的爬蟲演算法
    
    def show_algo(self):
        print(self.crawler_algo)
        
    def run(self, prod_name, data_num): # 執行多執行緒的爬蟲
        threads = []
        q = queue.Queue()
        
        for algo in self.crawler_algo:
            t = threading.Thread(target=algo, args=(q, prod_name, data_num))
            threads.append(t)
            
        for t in threads:
            t.start()
            
        for t in threads:
            t.join()
        
        prod_list = []
        
        while not q.empty():
            prod_list.append(q.get())
            
        print(prod_list)

In [ ]:
crawler = crawler(pchome_crawler, umall_crawler)
crawler.show_algo()

In [ ]:
crawler.run('ssd', 20)

# 測試ec_crawler module

In [1]:
from ec_crawler.algo import pchome, umall
from ec_crawler.engine import crawler

In [2]:
crawler = crawler(pchome, umall)
crawler.show_algo()

(<function pchome at 0x102a64048>, <function umall at 0x102bb8e18>)


In [3]:
crawler.run('ssd', 20)

<function pchome at 0x102a64048>
<function umall at 0x102bb8e18>
<Thread(Thread-4, initial)> thread start !
<Thread(Thread-5, initial)> thread start !
{'from': 'pchome', 'id': 'DRAH6C-A9008F54N', 'name': 'WD SSD 500GB 2.5吋 3D NAND固態硬碟(藍標)', 'prod_price': '2190', 'url': 'https://24h.pchome.com.tw/prod/DRAH6C-A9008F54N', 'prod_pic': 'https://d.ecimg.tw/items/DRAH6CA9008F54N/000002_1557821136.jpg'}
{'from': 'pchome', 'id': 'DRAH6C-A9008F563', 'name': 'WD SSD 1TB 2.5吋 3D NAND固態硬碟(藍標)', 'prod_price': '3980', 'url': 'https://24h.pchome.com.tw/prod/DRAH6C-A9008F563', 'prod_pic': 'https://d.ecimg.tw/items/DRAH6CA9008F563/000002_1557820146.jpg'}
{'from': 'pchome', 'id': 'DRAH5E-A9007CRTN', 'name': 'SanDisk SSD Plus 240GB 2.5吋SATAIII固態硬碟(G26)', 'prod_price': '988', 'url': 'https://24h.pchome.com.tw/prod/DRAH5E-A9007CRTN', 'prod_pic': 'https://d.ecimg.tw/items/DRAH5EA9007CRTN/000002_1488791017.jpg'}
{'from': 'pchome', 'id': 'DRAH5E-A9007CRVC', 'name': 'SanDisk SSD Plus 480GB 2.5吋SATAIII固態硬碟(G26)'

[{'from': 'umall',
  'id': '2176806',
  'name': 'Micron Crucial MX500 1TB  SSD',
  'prod_pic': 'https://media.u-mall.com.tw/NXimg/002176/2176806/2176806_LM.jpg?t=17292084173',
  'prod_price': '3749',
  'url': 'https://www.u-mall.com.tw//Micron-Crucial-MX500-1TB-SSD/i/2176806'},
 {'from': 'umall',
  'id': '1991796',
  'name': 'KISSDIAMOND 男女款 戶外超薄速乾透氣防曬耐刮機能褲/8色(S-3XL)',
  'prod_pic': 'https://media.u-mall.com.tw/NXimg/001991/1991796/1991796_LM.jpg?t=17439584831',
  'prod_price': '558',
  'url': 'https://www.u-mall.com.tw//KISSDIAMOND-男女款-戶外超薄速乾透氣防曬耐刮機能褲-8色-S-3XL/i/1991796'},
 {'from': 'umall',
  'id': '2319849',
  'name': 'SanDisk Ultra MicroSDXC 128GB A1/100MB/s 記憶卡 [公司貨]',
  'prod_pic': 'https://media.u-mall.com.tw/NXimg/002319/2319849/2319849_LM.jpg?t=17602109444',
  'prod_price': '499',
  'url': 'https://www.u-mall.com.tw//SanDisk-Ultra-MicroSDXC-128GB-A1-100MB-s-記憶卡-公司貨/i/2319849'},
 {'from': 'umall',
  'id': '2186856',
  'name': 'Micron Crucial P1 1TB SSD',
  'prod_pic': 'https://